In [1]:
import pandas as pd

df = pd.read_csv(
    filepath_or_buffer="prices_data/[SP500]_Daily_01-03-2020__01-03-2024.csv",
    delimiter="\t"
)

df = df.rename(
    columns = {
        '<DATE>':'date', 
        '<TIME>':'time', 
        '<OPEN>':'open', 
        '<HIGH>':'high', 
        '<LOW>':'low', 
        '<CLOSE>':'close'
    }
)

df = df.drop(['<TICKVOL>', '<VOL>', '<SPREAD>'], axis=1)

print(df)

            date     open     high      low    close
0     2020.03.02  2903.82  3093.92  2892.20  3071.83
1     2020.03.03  3076.48  3138.15  2975.46  2993.38
2     2020.03.04  2990.14  3130.41  2979.02  3111.44
3     2020.03.05  3109.86  3114.48  2999.13  3025.40
4     2020.03.06  3025.18  3038.54  2901.28  2963.15
...          ...      ...      ...      ...      ...
1031  2024.02.26  5084.76  5096.65  5065.45  5066.33
1032  2024.02.27  5066.71  5082.13  5056.85  5080.28
1033  2024.02.28  5079.76  5079.96  5053.42  5058.93
1034  2024.02.29  5063.23  5105.08  5051.60  5090.49
1035  2024.03.01  5092.02  5140.52  5080.72  5130.30

[1036 rows x 5 columns]


In [2]:
def heikinashi(df: pd.DataFrame) -> pd.DataFrame:
     
    df_HA = df.copy()
    df_HA['close']=(df_HA['open']+ df_HA['high']+ df_HA['low']+df_HA['close'])/4
 
    for i in range(0, len(df)):
        if i == 0:
            df_HA.loc[i, 'open']= ( (df_HA['open'][i] + df_HA['close'][i] )/ 2)
        else:
            df_HA.loc[i, 'open'] = ( (df_HA['open'][i-1] + df_HA['close'][i-1] )/ 2)
 
    df_HA['high']=df_HA[['open','close','high']].max(axis=1)
    df_HA['low']=df_HA[['open','close','low']].min(axis=1)
    return df_HA

dfHA = heikinashi(df)


In [3]:
from numpy import NaN

MA_WINDOW = 50

MA_serie = []

for i in range(0, len(df)):
    if i < MA_WINDOW:
        MA_serie.append(NaN)
    else:
        average = df["close"][i-MA_WINDOW:i].sum() / MA_WINDOW
        MA_serie.append(average)


dfHA["MA50"] = MA_serie
dfHA.index

dfHA["MA50_test"] = df["close"].rolling(window=50).mean()
dfHA


,date,open,high,low,close,MA50,MA50_test
0,2020.03.02,2947.131250,3093.920000,2892.200000,2990.4425,NaN,NaN
1,2020.03.03,2968.786875,3138.150000,2968.786875,3045.8675,NaN,NaN
2,2020.03.04,3007.327188,3130.410000,2979.020000,3052.7525,NaN,NaN
3,2020.03.05,3030.039844,3114.480000,2999.130000,3062.2175,NaN,NaN
4,2020.03.06,3046.128672,3046.128672,2901.280000,2982.0375,NaN,NaN
...,...,...,...,...,...,...,...
1031,2024.02.26,5053.279777,5096.650000,5053.279777,5078.2975,4856.8940,4863.9016
1032,2024.02.27,5065.788638,5082.130000,5056.850000,5071.4925,4863.9016,4871.1856
1033,2024.02.28,5068.640569,5079.960000,5053.420000,5068.0175,4871.1856,4877.6174
1034,2024.02.29,5068.329035,5105.080000,5051.600000,5077.6000,4877.6174,4884.1180


In [5]:
import plotly.graph_objects as go


dfpl = dfHA#[-300:-1]
fig = go.Figure(data=[go.Candlestick(x=dfpl.index,
                open=dfpl['open'],
                high=dfpl['high'],
                low=dfpl['low'],
                close=dfpl['close'],
                increasing_line_color= 'green', 
                decreasing_line_color= 'red')])

fig.add_trace(go.Scatter(x=dfpl.index, y=dfpl["MA50"], mode="lines", line=dict(color="#F50E0E"))) # RED
# fig.add_trace(go.Scatter(x=dfpl.index, y=dfpl["MA50_test"], mode="lines", line=dict(color="#1CF50E"))) # GREEN


for k in [4837.63, 4657.17, 4474.29, 4316.33, 4117.5, 3983.73, 3790.0, 3615.57, 3469.91, 3253.09, 2992.6]:
    fig.add_hline(y=k)


fig.update_layout(xaxis_rangeslider_visible=False)
fig.update_xaxes(showgrid=False)
fig.update_yaxes(showgrid=False)
fig.update_layout(paper_bgcolor='black', plot_bgcolor='black')

In [27]:
redOrGreen = []

for i in range(0, len(df)):
    if i < MA_WINDOW:
        redOrGreen.append(NaN)
    else:
        color = "green" if dfHA["open"][i] < dfHA["close"][i] else "red"
        redOrGreen.append(color)
    

dfHA["color"] = redOrGreen

In [49]:
# trades = pd.DataFrame({"succeed":[],"entry_price":[], "stop_loss":[], "take_profit":[]})
trades = pd.DataFrame(columns=["entry_date","exit_date", "entry_price", "stop_loss", "take_profit", "profit", "capital_after_trade"])
trades

,entry_date,exit_date,entry_price,stop_loss,take_profit,profit,capital_after_trade


In [50]:

CAPITAL = 1000 # $
maxRisk = 0.02 # in percent
# pipValueSP500ForEachLots = {"1" : 50, "0.1" : 5, "0.01" : 0.5}

# def getPipValueOfSP500AccordingTo(lot_size):
#     pipValue = lot_size*50 # en dollars US
#     return pipValue

def getSlInPipsForTrade(pipValue, lot_size):
    invested = CAPITAL*maxRisk
    sl_in_pips = invested/lot_size/pipValue
    return sl_in_pips

In [51]:

def between(a, min, max): 
    return min <= a and a <= max

in_position = False
entryPrice, sl, tp = 0, 0, 0
pipValue = 50
lot_size = 0.01
entry_date = df["date"][0]

for i in range(MA_WINDOW+3, len(df)):

    currentPrice = df["close"][i]

    if not in_position:
        ma50ZoneMin = dfHA["MA50"][i]-(df["close"][i]/100)*2
        ma50ZoneMax = dfHA["MA50"][i]+(df["close"][i]/100)*2

        isLast3CandlesInMA50Zone = False
        for j in range(1, 3+1):
            if between(dfHA["open"][j-1], ma50ZoneMin, ma50ZoneMax):
                isLast3CandlesInMA50Zone = True
                break
        
        if dfHA["MA50"][i] < dfHA["open"][i] and dfHA["color"][i] == "green" and isLast3CandlesInMA50Zone:
            entry_date = df["date"][i]
            entryPrice = currentPrice
            slInPips = getSlInPipsForTrade(pipValue, 0.01)
            # print(slInPips)
            sl = entryPrice-slInPips
            tp = entryPrice+slInPips
            in_position = True    
    else:
        lose = currentPrice <= sl
        win = tp <= currentPrice
        if lose or win:
            profit = slInPips*pipValue*lot_size if win else -(slInPips*pipValue*lot_size)
            CAPITAL += profit 
            new_trade = {"entry_date":entry_date, "exit_date":df["date"][i], "entry_price":entryPrice, "stop_loss":sl, "take_profit":tp, "profit":profit, "capital_after_trade":CAPITAL} 
            trades.loc[len(trades)] = new_trade
            in_position = False
   


In [52]:
trades

,entry_date,exit_date,entry_price,stop_loss,take_profit,profit,capital_after_trade
0,2020.06.16,2020.06.19,3127.26,3087.260000,3167.260000,-20.000000,980.000000
1,2020.06.23,2020.06.24,3123.39,3084.190000,3162.590000,-19.600000,960.400000
2,2020.06.30,2020.07.06,3095.74,3057.324000,3134.156000,19.208000,979.608000
3,2020.07.07,2020.07.10,3146.06,3106.875680,3185.244320,19.592160,999.200160
4,2020.07.13,2020.07.15,3157.58,3117.611994,3197.548006,19.984003,1019.184163
5,2020.07.16,2020.07.20,3204.47,3163.702633,3245.237367,20.383683,1039.567846
